In [1]:
%matplotlib inline
import pycistarget
pycistarget.__version__

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *

import pyranges as pr
from pycistarget.utils import *

In [2]:
## Defining paths to input/output and cistarget db
input_dir = '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/'
cistarget_db = '/lustre/scratch126/cellgen/team205/is10/fibroblasts/bulk_cytokines/cistarget/cistarget_db.regions_vs_motifs.rankings.feather'
motif_anno = '/nfs/team205/is10/resources/aerts_motifs/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl'
out_dir = '/lustre/scratch126/cellgen/team205/is10/fibroblasts/bulk_cytokines/cistarget/enrichment_padj0.01/'
os.makedirs(out_dir, exist_ok=True)

In [3]:
## Cutoffs for padj and logFC for DA peaks
padj_cutoff = 0.01
logFC_cutoff = 0.5

In [13]:
files = os.listdir(input_dir)
condition_paths = {}
for f in files:
    if ('differential' in f) and ('.csv' in f):
        condition_paths[f.split('.')[0]] = (input_dir + f)

In [14]:
condition_paths

{'differential_peak_table_FDR05_annotated_matched_Cytokine': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Cytokine.csv',
 'differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat.csv',
 'differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine.csv'}

In [23]:
peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] >= logFC_cutoff)]

,seqnames,start,end,width,strand,logFC,logCPM,F,num.tests,num.up.logFC,...,geneEnd,geneLength,geneStrand,geneId,transcriptId,distanceToTSS,ENSEMBL,SYMBOL,GENENAME,atac_peak
7,chr1,81440818,81441170,353,*,3.201871,-0.762567,59.333780,1,1,...,81484722,70448,1,23266,ENST00000674458.1,26543,ENSG00000117114,ADGRL2,adhesion G protein-coupled receptor L2,NaN
37,chr10,123857827,123858764,938,*,1.929552,0.941050,46.984255,1,1,...,123887166,6958,2,119587,ENST00000481775.1,28402,ENSG00000121898,CPXM2,"carboxypeptidase X, M14 family member 2","chr10:123858287-123858787,CPXM2,Intronic"
88,chr6,29516141,29516601,461,*,1.863218,0.398044,41.502718,1,1,...,29531240,2520,1,100507362,ENST00000658484.1,-12120,ENSG00000224582,LINC01015,long intergenic non-protein coding RNA 1015,"chr6:29516029-29516529,LINC01015,Distal"
505,chr8,58220489,58221914,1426,*,1.851944,-0.068502,33.942401,3,3,...,58243984,16407,2,107986945,ENST00000671135.1,22070,NaN,LOC107986945,uncharacterized LOC107986945,"chr8:58220688-58221188,LOC101929528,Distal"
558,chr8,58169465,58170306,842,*,1.655663,0.106460,31.250404,1,1,...,58243984,16407,2,107986945,ENST00000671135.1,73678,NaN,LOC107986945,uncharacterized LOC107986945,"chr8:58169784-58170284,LOC101929528,Intronic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45956,chr16,70467894,70468771,878,*,1.188838,-0.973321,11.124512,2,2,...,70479681,5057,1,197258,ENST00000498702.1,-5854,ENSG00000157353,FCSK,fucose kinase,NaN
45969,chrX,144566744,144567028,285,*,1.445536,-1.636126,9.841566,1,1,...,145256208,8706,1,494118,ENST00000370493.4,-680475,ENSG00000203923,SPANXN1,SPANX family member N1,NaN
45971,chr7,157517944,157518938,995,*,1.331460,-1.202676,11.874881,3,1,...,157518658,2866,1,105375610,ENST00000648425.1,2151,NaN,LOC105375610,uncharacterized LOC105375610,NaN
45973,chr17,79117525,79117755,231,*,1.183959,-1.094037,9.840694,1,1,...,79115580,26214,2,146713,ENST00000648001.1,-1945,ENSG00000167281,RBFOX3,RNA binding fox-1 homolog 3,NaN


In [25]:
region_sets = {}

for cond in condition_paths.keys():
    peaks = pd.read_csv(condition_paths[cond])
    ## Up
    up = peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] >= logFC_cutoff)]
    up_peaks=(up['seqnames'] + ":" +up['start'].astype(str) + '-' + up['end'].astype(str)).tolist()

    region_sets[(cond + '_up')] = pr.PyRanges(region_names_to_coordinates(up_peaks))

    ## Down
    down = peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] < ((-1) * logFC_cutoff))]
    down_peaks=(down['seqnames'] + ":" +down['start'].astype(str) + '-' + down['end'].astype(str)).tolist() 

    region_sets[(cond + '_down')] = pr.PyRanges(region_names_to_coordinates(down_peaks))

In [26]:
region_sets

{'differential_peak_table_FDR05_annotated_matched_Cytokine_up': +--------------+-----------+-----------+
 | Chromosome   | Start     | End       |
 | (category)   | (int64)   | (int64)   |
 |--------------+-----------+-----------|
 | chr1         | 81440818  | 81441170  |
 | chr1         | 209720330 | 209721193 |
 | chr1         | 81424081  | 81424543  |
 | chr1         | 81414458  | 81414870  |
 | ...          | ...       | ...       |
 | chrX         | 77362944  | 77363228  |
 | chrX         | 151741713 | 151742023 |
 | chrX         | 135225097 | 135225327 |
 | chrX         | 144566744 | 144567028 |
 +--------------+-----------+-----------+
 Unstranded PyRanges object has 1,107 rows and 3 columns from 23 chromosomes.
 For printing, the PyRanges was sorted on Chromosome.,
 'differential_peak_table_FDR05_annotated_matched_Cytokine_down': +--------------+-----------+-----------+
 | Chromosome   | Start     | End       |
 | (category)   | (int64)   | (int64)   |
 |--------------+--------

In [27]:
# Run
cistarget_dict = run_cistarget(ctx_db = cistarget_db,
                               region_sets = region_sets,
                               specie = 'homo_sapiens',
                               annotation_version = 'v10nr_clust',
                               path_to_motif_annotations = motif_anno,
                               auc_threshold = 0.005,
                               nes_threshold = 3.0,
                               rank_threshold = 0.05,
                               annotation = ['Direct_annot', 'Orthology_annot'],
                               n_cpu = 1,
                               _temp_dir='/lustre/scratch126/cellgen/team205/is10/tmp/')

2023-11-28 22:05:22,446 cisTarget    INFO     Reading cisTarget database
2023-11-28 22:07:15,178 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_up which has 1356 regions
2023-11-28 22:07:43,761 cisTarget    INFO     Annotating motifs for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-11-28 22:07:47,842 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-11-28 22:07:48,024 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_down which has 57254 regions
2023-11-28 22:08:03,550 cisTarget    INFO     No enriched motifs found for differential_peak_table_FDR05_annotated_matched_Cytokine_down
2023-11-28 22:08:03,576 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up which has 1682 regions
2023-11-28 22:08:18,098 cisTarget    INFO     Annotating motifs f

In [28]:
# Save
import pickle
with open((out_dir + 'cistarget_dict_NES3.0.pkl'), 'wb') as f:
  pickle.dump(cistarget_dict, f)

In [29]:
## Also exporting results in a table format:
motif_enrichment_dict = {key: cistarget_dict[key].motif_enrichment for key in cistarget_dict.keys()}
motif_enrichment_table=pd.concat([motif_enrichment_dict[key] for key in motif_enrichment_dict.keys()], axis=0, sort=False)
motif_enrichment_table.to_csv((out_dir + "cistarget_res_df_NES3.0.txt"), sep="\t")

In [30]:
cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Cytokine_up")

,Logo,Region_set,Direct_annot,Orthology_annot,NES,AUC,Rank_at_max,Motif_hits
predrem__nrMotif404,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.834570,0.009876,3258.0,33
predrem__nrMotif1203,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.817860,0.009853,11831.0,82
cisbp__M00569,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.567846,0.009503,15087.0,90
jaspar__MA0249.2,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,"TCF15, FERD3L, TCF21, TCF23, FIGLA, TWIST1, TCF24, HAND2, TWIST2, MSC, BHLHA9, HAND1, SCX",5.440159,0.009325,4635.0,41
cisbp__M00127,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.324768,0.009164,4357.0,34
flyfactorsurvey__CG16778_SANGER_5_FBgn0003715,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.279683,0.009101,6091.0,56
hdpi__NR4A1,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NR4A1,NaN,5.061827,0.008796,9890.0,72
cisbp__M00336,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,PAX6,NaN,5.056782,0.008789,16127.0,99
cisbp__M00748,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.040703,0.008767,4255.0,36
hdpi__USF1,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,USF1,NaN,5.007599,0.008720,1596.0,19


In [32]:
cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Cytokine_down")

,Logo,Region_set,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot,NES,AUC,Rank_at_max


In [33]:
# Also running a permissive analysis (NES > 0.1 instead of 3) to be used for more quantitative comparisons between classes
cistarget_dict_perm = run_cistarget(ctx_db = cistarget_db,
                               region_sets = region_sets,
                               specie = 'homo_sapiens',
                               annotation_version = 'v10nr_clust',
                               path_to_motif_annotations = motif_anno,
                               auc_threshold = 0.005,
                               nes_threshold = 0.1,
                               rank_threshold = 0.05,
                               annotation = ['Direct_annot', 'Orthology_annot'],
                               n_cpu = 1,
                               _temp_dir='/lustre/scratch126/cellgen/team205/is10/tmp/')

2023-12-03 17:26:27,935 cisTarget    INFO     Reading cisTarget database
2023-12-03 17:27:54,715 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_up which has 1356 regions
2023-12-03 17:28:54,828 cisTarget    INFO     Annotating motifs for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-03 17:29:06,079 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-03 17:29:18,968 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_down which has 57254 regions
2023-12-03 17:45:05,613 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_down
2023-12-03 17:45:41,224 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up which has 1682 regions
2023-12-03 17:47:32,044 cisTarget    INFO     Annotating motifs for diff

In [34]:
# Save
import pickle
with open((out_dir + 'cistarget_dict_NES0.1.pkl'), 'wb') as f:
  pickle.dump(cistarget_dict_perm, f)

In [35]:
## Also exporting results in a table format:
motif_enrichment_dict_perm = {key: cistarget_dict_perm[key].motif_enrichment for key in cistarget_dict_perm.keys()}
motif_enrichment_table_perm=pd.concat([motif_enrichment_dict_perm[key] for key in motif_enrichment_dict_perm.keys()], axis=0, sort=False)
motif_enrichment_table_perm.to_csv((out_dir + "cistarget_res_df_NES0.1.txt"), sep="\t")